In [1]:
%pip install transformers torch accelerate datasets evaluate numpy pandas scikit-learn -r requirements.txt

  Obtaining dependency information for flask~=3.0.3 from https://files.pythonhosted.org/packages/61/80/ffe1da13ad9300f87c93af113edd0638c75138c42a0994becfacac078c06/flask-3.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for Werkzeug>=3.0.0 from https://files.pythonhosted.org/packages/4b/84/997bbf7c2bf2dc3f09565c6d0b4959fefe5355c18c4096cfd26d83e0785b/werkzeug-3.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for itsdangerous>=2.1.2 from https://files.pythonhosted.org/packages/04/96/92447566d16df59b2a776c0fb82dbc4d9e07cd95062562af01e408583fc4/itsdangerous-2.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker>=1.6.2 from https://files.pythonhosted.org/packages/bb/2a/10164ed1f31196a2f7f3799368a821765c62851ead0e630ab52b8e14b4d0/blinker-1.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 19.1 MB/s eta 0:00:00
Note: y

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
isTraining = True

In [3]:
import pandas as pd
import cleaning_tool as ct

# Load your dataset
df = pd.read_csv('training_set.csv')
df = df.dropna()
df = ct.filter_single_users(dataframe=df, min_pull=5)
print(df.head())


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/SA24-G2/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/SA24-G2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   number                                              title  assignee  \
0       3              Omnisharp not included in Linux build   jrieken   
1       4      C# bracket insertion, indentation not working  alexdima   
2       5                        welcome.md packaged wrongly   isidorn   
3       6  Windows only - OmniSharp does not provide Inte...   jrieken   
4       8        Flash between opening of workspaces, reload   jrieken   

                                                body  
0  our linux build machin includ csharp-o/**bin**...  
1            type ( expect => close ) actual => noth  
2  vscode > help > show welcom > work if i look c...  
3  upgrad rc1 dnx/runtim ``` bash git clone https...  
4  start code set dark theme _other default dark ...  


In [29]:
from sklearn.preprocessing import LabelEncoder

# Encode the assignee names
label_encoder = LabelEncoder()
df['assignee_encoded'] = label_encoder.fit_transform(df['assignee'])

df['input_text'] = "<#TITLE-START#> " + df['title'] + " <#TITLE-END#> <#BODY-START#> " + df['body'] + " <#BODY-END#>"


# Split into input features (titles) and labels (encoded assignees)
titles = df['input_text'].tolist()

trainingSet = df[df['number'] < 185000]['input_text'].tolist()
evaluationSet = df[ (185000 <= df['number']) & (df['number']< 210000)]['input_text'].tolist()
testSet = df[ 21000 <= df['number']]['input_text'].tolist()

labels = df['assignee_encoded'].tolist()

trainingLabels = df[df['number'] < 185000]['assignee_encoded'].tolist()
evaluationLabels = df[ (185000 <= df['number']) & (df['number']< 210000)]['assignee_encoded'].tolist()
testLabels = df[ 21000 <= df['number']]['assignee_encoded'].tolist()

print(len(trainingSet), len(evaluationSet), len(testSet))



110305 11926 111434


In [27]:
from transformers import AutoTokenizer

# Choose a model, e.g., 'distilbert-base-uncased'
# model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the input titles
# inputs = tokenizer(titles, padding=True, truncation=True, return_tensors='pt', max_length=128)
# inputs = tokenizer(titles, padding=True, truncation=True, return_tensors='pt', max_length=128).to("cuda")


/home/SA24-G2/bug-triaging/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
inputs

{'input_ids': tensor([[  101,  1026,  1001,  ...,  2378,  1026,   102],
        [  101,  1026,  1001,  ...,     0,     0,     0],
        [  101,  1026,  1001,  ...,     0,     0,     0],
        ...,
        [  101,  1026,  1001,  ...,  4468, 13675,   102],
        [  101,  1026,  1001,  ...,  2015,  1012,   102],
        [  101,  1026,  1001,  ...,  1012, 21035,   102]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}

In [30]:
from datasets import Dataset

def makeDataset(current, labels):
    inputs = tokenizer(current, padding=True, truncation=True, return_tensors='pt', max_length=128).to("cuda")
    
    return Dataset.from_dict({
    'input_ids': inputs['input_ids'],
    'attention_mask': inputs['attention_mask'],
    'labels': labels
})





trainingSet = makeDataset(trainingSet, trainingLabels)
evaluationSet =  makeDataset(evaluationSet, evaluationLabels)
testSet =  makeDataset(testSet, testLabels)

# # Split the dataset into training and validation sets
# train_test = dataset.train_test_split(test_size=0.2)
# train_dataset = train_test['train']
# test_dataset = train_test['test']

In [31]:
import evaluate
import numpy as np

# Load metric functions
accuracy_metric = evaluate.load('accuracy')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load('f1')

# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted', zero_division=0)
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    
    return {
        'accuracy': accuracy['accuracy'],
        'precision': precision['precision'],
        'recall': recall['recall'],
        'f1': f1['f1'],
    }
    

In [32]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Define training arguments with early stopping and final model saving
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy='epoch',  # Save at the end of each epoch
    save_total_limit=1,  # Keep only the most recent model
    load_best_model_at_end=True,  # Automatically load the best model at the end
    metric_for_best_model='eval_loss',  # Use validation loss to select the best model
    greater_is_better=False,  # Lower loss is better
)

# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainingSet,
    eval_dataset=evaluationSet,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Early stopping with patience
)

# Train the model
trainer.train()



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/SA24-G2/bug-triaging/.venv/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [33]:
trainer.evaluate(testSet)


KeyboardInterrupt: 

## Reload model from file

In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import json

model = AutoModelForSequenceClassification.from_pretrained('./results/checkpoint-18759')
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
with open("labels_json.json") as f:
    labelsFromJson = json.load(f)
isTraining = False

## Model evaluation

In [16]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy='epoch',  # Save at the end of each epoch
    save_total_limit=1,  # Keep only the most recent model
    load_best_model_at_end=True,  # Automatically load the best model at the end
    metric_for_best_model='eval_loss',  # Use validation loss to select the best model
    greater_is_better=False,  # Lower loss is better
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],  # Early stopping with patience
)

trainer.evaluate()

/home/SA24-G2/bug-triaging/.venv/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


{'eval_loss': 3.283426284790039,
 'eval_model_preparation_time': 0.0044,
 'eval_accuracy': 0.133781136300028,
 'eval_precision': 0.10409391825826096,
 'eval_recall': 0.133781136300028,
 'eval_f1': 0.05157461346601556,
 'eval_runtime': 32.682,
 'eval_samples_per_second': 765.283,
 'eval_steps_per_second': 11.964}

## CUDA

In [15]:
import torch
from scipy.special import softmax

# Check if CUDA is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Move the model to the correct device
model.to(device)

def predict_assignee(title, body):
    # Concatenate title and body
    combined_input = title + " " + body
    
    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors='pt', padding=True, truncation=True, max_length=128)
    
    # Move input tensors to the correct device
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Get model output
    outputs = model(**inputs)
    
    # Get logits
    logits = outputs.logits.detach().cpu().numpy()[0]
    
    # Get probabilities using softmax
    probabilities = softmax(logits)
    
    # Create a list of (assignee, probability) pairs
    if isTraining:
        assignee_probs = list(zip(label_encoder.classes_, probabilities))
    else:
        assignee_probs = list(zip(labelsFromJson, probabilities))
    
    # Sort by probability in descending order
    ranked_assignees = sorted(assignee_probs, key=lambda x: x[1], reverse=True)
    
    return ranked_assignees

# Example prediction with both title and body
ranked_list = predict_assignee("Side bar always showing when opening VS Code", "When opening VS Code, the sidebar is always visible")

for index, (assignee, probability) in enumerate(ranked_list):
    print(f"{index + 1}: {assignee}: {probability:.2f}")


1: mjbvz: 0.09
2: sandy081: 0.07
3: Tyriar: 0.07
4: bpasero: 0.06
5: joaomoreno: 0.06
6: jrieken: 0.05
7: isidorn: 0.05
8: aeschli: 0.04
9: roblourens: 0.04
10: alexdima: 0.04
11: rebornix: 0.03
12: meganrogge: 0.02
13: deepak1556: 0.02
14: weinand: 0.02
15: chrmarti: 0.02
16: connor4312: 0.02
17: alexr00: 0.02
18: TylerLeonhardt: 0.01
19: sbatten: 0.01
20: lszomoru: 0.01
21: hediet: 0.01
22: ramya-rao-a: 0.01
23: dbaeumer: 0.01
24: rzhao271: 0.01
25: JacksonKearl: 0.01
26: RMacfarlane: 0.01
27: lramos15: 0.01
28: andreamah: 0.01
29: amunger: 0.01
30: benibenj: 0.01
31: joyceerhl: 0.01
32: egamma: 0.01
33: octref: 0.01
34: miguelsolorio: 0.01
35: Yoyokrazy: 0.01
36: chrisdias: 0.01
37: bhavyaus: 0.01
38: aiday-mar: 0.00
39: kieferrm: 0.00
40: tsalinger: 0.00
41: eamodio: 0.00
42: MeghanKulkarni: 0.00
43: Lixire: 0.00
44: daviwil: 0.00
45: DonJayamanne: 0.00
46: justschen: 0.00
47: ulugbekna: 0.00
48: gushuro: 0.00
49: waderyan: 0.00
50: hbons: 0.00
51: karrtikr: 0.00
52: stevencl: 0.00

## MACOS

In [3]:
import torch
from scipy.special import softmax

# Check if MPS is available
device = torch.device('mps') if torch.has_mps else torch.device('cpu')

# Move the model to the correct device
model.to(device)

# def predict_assignee(title):
#     # Tokenize the input
#     inputs = tokenizer(title, return_tensors='pt', padding=True, truncation=True, max_length=128)

#     # Move input tensors to the correct device
#     inputs = {key: value.to(device) for key, value in inputs.items()}

#     # Get model output
#     outputs = model(**inputs)

#     # Get the predicted class
#     predicted_class = outputs.logits.argmax(dim=1).item()

#     # Return the assignee name
#     return label_encoder.inverse_transform([predicted_class])[0]

# Example prediction
# print(predict_assignee("Change the name"))

def predict_assignee(title, body):
    # Concatenate title and body
    combined_input = title + " " + body
    
    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors='pt', padding=True, truncation=True, max_length=128)
    
    # Move input tensors to the correct device
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Get model output
    outputs = model(**inputs)
    
    # Get logits
    logits = outputs.logits.detach().cpu().numpy()[0]
    
    # Get probabilities using softmax
    probabilities = softmax(logits)
    
    # Create a list of (assignee, probability) pairs
    if isTraining:
        assignee_probs = list(zip(label_encoder.classes_, probabilities))
    else:
        assignee_probs = list(zip(labelsFromJson, probabilities))
    
    # Sort by probability in descending order
    ranked_assignees = sorted(assignee_probs, key=lambda x: x[1], reverse=True)
    
    return ranked_assignees

# Example prediction with both title and body
ranked_list = predict_assignee("Side bar always showing when opening VS Code", "When opening VS Code, the sidebar is always visible")

for index, (assignee, probability) in enumerate(ranked_list):
    print(f"{index + 1}: {assignee}: {probability:.2f}")



/tmp/ipykernel_4123887/1114867945.py:5: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device('mps') if torch.has_mps else torch.device('cpu')


NameError: name 'label_encoder' is not defined